In [ ]:
%matplotlib inline

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math
from pymoo.factory import get_performance_indicator
from itertools import product
from palettable.scientific.diverging import Vik_7

pd.set_option('mode.chained_assignment', None)
pd.set_option('display.float_format', lambda x: '%.4f' % x)
plt.style.use(['science', 'ieee'])


In [ ]:
results_list = ["results-database-20220321153434.json"]

from tinydb import TinyDB, Query

dataset = pd.DataFrame()

for results in results_list:
    database = TinyDB("results/" + results)
    q = Query()
    temp_results = database.all()
    # temp_results=database.search(q.paradigms ==['ECS-CLB'])

    temp = pd.DataFrame(temp_results)
    dataset = pd.concat([dataset, temp])
dataset = dataset[[
    "system_accuracies", "points","paradigms"
]]
dataset = dataset.fillna(0)

dataset

In [ ]:
h_dataset = dataset[['points', 'paradigms']]
h_dataset = list(h_dataset.to_dict('records'))

heuristic_dataset=[]
for item in h_dataset:
    points = item.pop('points')
    for point in points:
        point.update({'policy':str(item['paradigms'])})
        heuristic_dataset.append(point)

he_dataset = heuristic_dataset

df = pd.DataFrame(he_dataset).drop_duplicates(subset=None,
                                              keep='first',
                                              inplace=False)

with plt.rc_context({"legend.labelspacing": 0.01,"legend.handletextpad": 0}):
    temp_df=df.rename(columns={'average_accuracy':'average accuracy',
                            'bandwidth_reserved_ratio': 'bandwidth reserved'
                                })
    from palettable.scientific.diverging import Vik_7
    import matplotlib.ticker as mtick
    fig, axes = plt.subplots(1,
                             3,
                             gridspec_kw={
                                 'width_ratios': [1, 1, 1],
                                 "wspace": 0.05,
                             },
                             sharex=True,
                             sharey=True,
                             figsize=(9, 2))

    for index,policy in enumerate(temp_df['policy'].unique()):
        tmp_filter= temp_df['policy'] == policy
        g = sns.scatterplot(data=temp_df.where(tmp_filter),
                        x="served",
                        y="average accuracy",
                        hue="bandwidth reserved",
                        s=8,
                        ax=axes[index],
                        legend= None if index!=2 else "brief",
                        palette=Vik_7.mpl_colormap)
    deployments= [
        "\\textbf{SP}",
        "\\textbf{EP+CP}",
        "\\textbf{EP+CP+SP}",
    ]
    for index,ax in enumerate(axes):
        if index==0:
            ax.set_ylabel("$f_A$")
        ax.set_xlabel("$f_N$")
        ax.set_title(deployments[index])
        ax.yaxis.set_tick_params(labelsize='small')
        ax.xaxis.set_tick_params(labelsize='small')
        ax.title.set_size('small')
        ax.tick_params(axis='x',which="major",width=1)
        ax.tick_params(axis='y',which="major",width=1)
        ax.spines['left'].set_linewidth(1)
        ax.spines['right'].set_linewidth(1)
        ax.spines['top'].set_linewidth(1)
        ax.spines['bottom'].set_linewidth(1)
        ax.set_ylim(0.64, 0.915)
        if index==2:
            axright = ax.twinx()
            axright.yaxis.label.set_text("$f_A$")
            axright.yaxis.label.set_color("white")
            axright.yaxis.set_tick_params(labelsize='small',labelcolor="white")
            axright.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.2f'))
            axright.tick_params(axis='y', which="major", width=1)
            axright.set_ylim(0.64, 0.915)


    axes[-1].get_legend().set_title(
        "$f_B {\scriptstyle(\\times \\text{100Mbps})}$")

    for lh in axes[-1].get_legend().legendHandles:
        lh._sizes = [8]

    # plt.tight_layout()

    plt.savefig("figures/illustration-pareto.pdf",dpi=300)

In [ ]:
results = pd.DataFrame()
experiments = dataset.to_dict('records')
item = experiments[-1]
data = []
query = item
points = query['points']
systems = query['system_accuracies']
zipped = zip(points, systems)

temp_pairs = sorted(zipped, key=lambda _: -_[0]['average_accuracy'])
temp = list(map(lambda _: _[1], temp_pairs))

for objective in ["number","accuracy","bandwidth"]:
    data = []
    for number, record in enumerate(temp):
        if objective =="number":
            metric=str(temp_pairs[number][0]['served']).zfill(2)
        elif objective =="accuracy":
            metric= '{:.3f}'.format(
                round(temp_pairs[number][0]['average_accuracy'], 3))
        else:
            metric= '{:.3f}'.format(
                round(temp_pairs[number][0]['bandwidth_reserved_ratio'], 3))
        result = list(
            map(lambda x: dict(objective=objective,metric=metric, accuracy='{:.4f}'.format(x)),
                record))
        data.extend(result)
    if len(data) > 0:
        data = pd.DataFrame(data)
        data = data.groupby(["objective","metric",
                             "accuracy"]).size().reset_index(name='counts')
        data = data.drop_duplicates()
        results = pd.concat([results, data])

edge_accuracies = [
    "0.6250", "0.6500", "0.6750", "0.7000", "0.7250", "0.7500", "0.7750",
    "0.8000"
]
cloud_accuracies = ["0.9000"]

synergy_accuracies = [
    "0.6525", "0.6750", "0.6975", "0.7200", "0.7425", "0.7650", "0.7875",
    "0.8100"
]

accuracies = set(edge_accuracies + cloud_accuracies + synergy_accuracies)

from palettable.colorbrewer.sequential import Blues_9
from palettable.scientific.diverging import Vik_7
from matplotlib.colors import LogNorm


def facet(data, color, vmin=None, vmax=None, cbar_ax=None):
    temp = []
    for metric in data['metric'].unique():
        temp_data = data[data['metric'] == metric]
        curr_accuracies = set(temp_data['accuracy'].unique())
        delta = accuracies - curr_accuracies
        for accuracy in delta:
            temp.append(dict(metric=metric, accuracy=accuracy, counts=np.nan))

    data = pd.concat([data, pd.DataFrame(temp)])
    data = data.sort_values(['metric', 'accuracy'],ascending=True)
    data = data.pivot("metric", "accuracy", "counts")
    data=data.fillna(0.99)
    g = sns.heatmap(data,
                    xticklabels=1,
                    cmap=Blues_9.mpl_colormap,
                    cbar_ax=cbar_ax,
                    # linewidth=0.001,
                    vmin=vmin,
                    vmax=vmax,
                    norm=LogNorm(),
                    cbar_kws={
                        'label': None
                    }
                    )
    g.set_xticklabels(g.get_xticklabels(), rotation=90, fontsize="xx-small")
    g.set_yticklabels(g.get_yticklabels(), rotation=0, fontsize="xx-small")
    cbar = g.collections[0].colorbar
    cbar.ax.tick_params(labelsize='xx-small', which="major")

    g.title.set_size('small')
    g.tick_params(axis='x', which="major", width=0)
    g.tick_params(axis='y', which="major", width=0)
    g.tick_params(axis='x', which="minor", width=0)
    g.tick_params(axis='y', which="minor", width=0)


g = sns.FacetGrid(results,
                  col="objective",
                  sharey=False,
                  sharex=True,
                  margin_titles=True,
                  gridspec_kws={"wspace":0.25},
                  height=4,
                  aspect=0.4)
vmin = 0
vmax = results["counts"].max()

cbar_ax = g.fig.add_axes([0.91, 0.1, 0.01, .79])
g.map_dataframe(facet, vmin=vmin, vmax=vmax,
                cbar_ax=cbar_ax
                )
g.set_titles(col_template='{col_name}')

titles=[r"$f_N$",r"$f_A$",r"$f_B$"]

for index in [0,1,2]:
    g.axes[0, index].set_xlabel(r'$a_s$',fontsize="small")
    g.axes[0, index].set_title(titles[index], fontsize="small")

    for tick in g.axes[0, index].get_xticklabels():
        if tick.get_text() in edge_accuracies and tick.get_text() in synergy_accuracies:
            tick.set_color( Vik_7.mpl_colors[-2])
        else:
            if tick.get_text() in edge_accuracies:
                tick.set_color( Vik_7.mpl_colors[-1])
            elif tick.get_text() in cloud_accuracies:
                tick.set_color("black")
            else:
                tick.set_color(Vik_7.mpl_colors[0])

plt.savefig("figures/illustration.pdf",dpi=300)